In [1]:
from sys import path
path.append('..//modules')
import psycopg2 as pg
from geoalchemy2 import Geometry, WKTElement
from sqlalchemy import create_engine, MetaData, Table
import pandas.io.sql as psql
import pandas as pd
import time
#create a postgres engine and a pg connection
from modules.db_connector import *
print("DB connecting engine imported")

DB connecting engine imported


In [3]:
#import module for call arcgis map server and transforming feauture layer to gpdf (shoutout to Whiskers,2020)
from modules import call_arcgis_rest_service
print("Module call_arcgis_rest_service imported")

start = time.time()
print("Making to call ArcGis mapserver")
#extract layer from Arcgis rest service and transform to a gpdf containing all SGS (it takes about 2 minutes)
gpdf_from_arcgis = call_arcgis_rest_service.call_arcgis_rest_to_gpdf("https://gis.geology.sk/arcgis/rest/services/wgs_naucnaGeologia/VGL_wgs/MapServer/")
print("gpdf_from_arcgis retrieved in : ", round(time.time() - start, 2), " seconds.")

#load the created gpdf in to postgis db
gpdf_from_arcgis.to_postgis('sk_sgs_vgl_sguds', engine, schema = 'gtlab', if_exists='replace', index=False, 
                   dtype={'geometry': Geometry(geometry_type='Point', srid= '3857')})

print("gpdf_from_arcgis loaded in db under name sk_sgs_vgl_sguds in: ", round(time.time() - start, 2), " seconds.")



Module call_wfs imported
Making to call ArcGis mapserver
gpdf_from_arcgis retrieved in :  3.88  seconds.
gpdf_from_arcgis loaded in db under name sk_sgs_vgl_sguds in:  4.03  seconds.
Control atribute for scrapping added 4.03  seconds.


C:\Users\csb\anaconda\Lib\site-packages\geopandas\io\sql.py:415: UserWarning: Could not parse CRS from the GeoDataFrame. Inserting data without defined CRS.
  srid = _get_srid_from_crs(gdf)


In [6]:
#A FEW ADDITIONS AND UPDATES TO sk_sgs_vgl_sguds 
with engine.connect() as con1:
    alter_stmt1 = "ALTER TABLE gtlab.sk_sgs_vgl_sguds ADD COLUMN details_scrapped text; "
    con1.execute(text(alter_stmt1))
    con1.commit()
    print("Control atribute for scrapping added")
    alter_stmt2 = 'ALTER TABLE gtlab.sk_sgs_vgl_sguds RENAME COLUMN "geometry" to geom; '
    con1.execute(text(alter_stmt2))
    con1.commit()
    print("Geometry columns renamed to geom")
    update_stmt1 = "SELECT UpdateGeometrySRID('gtlab','sk_sgs_vgl_sguds', 'geom', 3857);"
    con1.execute(text(update_stmt1))
    print("SRID UPDATED TO 3857")
    con1.commit()
    alter_stmt4 = 'ALTER TABLE gtlab.sk_sgs_vgl_sguds ADD COLUMN gku_municipality_id integer; '
    con1.execute(text(alter_stmt4))
    con1.commit()
    print("Column gku_municipality_id added")
    alter_stmt4 = 'ALTER TABLE gtlab.sk_sgs_vgl_sguds ADD COLUMN dmo_id integer; '
    con1.execute(text(alter_stmt4))
    con1.commit()
    print("Column dmo_id added")
    con1.close()



Geometry columns renamed to geom
SRID UPDATED TO 3857


In [10]:
from modules import scrappSGUDS_vgl24

with engine.connect() as con1:
    q = "select * from gtlab.sk_sgs_vgl_sguds where details_scrapped is null order by id"
    df_vgl = pd.read_sql(q, con=con1)
    con1.close()
    
scrappSGUDS_vgl24.py.get_vgl_web_app_to_sgs_df(df_vgl)

ProgrammingError: (psycopg2.errors.SyntaxError) syntax error at or near "where"
LINE 1: ...ipalities_gku WHERE sk_sgs_vgl_sguds.id = 1.0 AND where st_w...
                                                             ^

[SQL: SELECT sk_sgs_vgl_sguds.id, sk_municipalities_gku."Kód_obce" FROM gtlab.sk_sgs_vgl_sguds, gtlab.sk_municipalities_gku WHERE sk_sgs_vgl_sguds.id = 1.0 AND where st_within(st_transform(sk_sgs_vgl_sguds.geom, 4326), st_transform(sk_municipalities_gku.geom, 4326 ));]
(Background on this error at: https://sqlalche.me/e/20/f405)

In [7]:
from modules import get_municipality_id_sgs_vgl

with engine.connect() as con1:
    q = "select * from gtlab.sk_sgs_vgl_sguds where gku_municipality_id is null order by id"
    df_vgl = pd.read_sql(q, con=con1)
    con1.close()
    
get_municipality_id_sgs_vgl.get_municipality_id(df_vgl)

1.0 SK0428514578
2.0 SK0329514861
3.0 SK0416523437
4.0 SK0214507342
5.0 SK0323509043
6.0 SK0316512478
7.0 SK0316512419
8.0 SK032C517097
9.0 SK0321580244
10.0 SK0417524301
11.0 SK032D517046
12.0 SK0416560103
13.0 SK0315510386
14.0 SK0428526266
15.0 SK0428525634
16.0 SK0426518123
17.0 SK0107508179
18.0 SK0104529389
19.0 SK0428525634
20.0 SK0428526053
21.0 SK0428526312
22.0 SK0428526266
23.0 SK0428525863
24.0 SK0426521337
25.0 SK0428526231
26.0 SK0426518123
27.0 SK0227513881
28.0 SK0417525383
29.0 SK0235504092
30.0 SK0236505421
31.0 SK042A526657
32.0 SK0106504637
33.0 SK0217556688
34.0 SK0329514811
35.0 SK0217556513
36.0 SK041A527092
37.0 SK0326511323
38.0 SK0104529371
39.0 SK0222557625
40.0 SK0313510211
41.0 SK0318510637
42.0 SK031A510106
43.0 SK0313509957
44.0 SK0416580368
45.0 SK031B517402
46.0 SK0106507831
47.0 SK0106507831
48.0 SK0106508021
49.0 SK0321508748
50.0 SK0106508161
51.0 SK0107508179
52.0 SK0107508179
53.0 SK0328526142
54.0 SK0428525529
55.0 SK042A543519
56.0 SK0315510858
5

438.0 SK032D516589
439.0 SK0329514837
440.0 SK0329514811
441.0 SK0329514811
442.0 SK0326511391
443.0 SK0427523259
444.0 SK032D516848
445.0 SK032D516660
446.0 SK0322517160
447.0 SK032B518891
448.0 SK0237500658
449.0 SK032D517267
450.0 SK0324518492
451.0 SK0324518468
452.0 SK032C516902
453.0 SK032C517291
454.0 SK032C517291
455.0 SK0319512273
456.0 SK032B518891
457.0 SK032D517267
458.0 SK0321508438
459.0 SK0232502693
460.0 SK0227513881
461.0 SK032B518204
462.0 SK0329514811
463.0 SK0426522007
464.0 SK041D529079
465.0 SK0426521604
466.0 SK042B528315
467.0 SK0227513997
468.0 SK0227514063
469.0 SK0315511188
470.0 SK0417525260
471.0 SK0414581640
472.0 SK0416523852
473.0 SK0328525987
474.0 SK0316512648
475.0 SK0428526096
476.0 SK0419520829
477.0 SK0417524140
478.0 SK0316512079
479.0 SK0226513601
480.0 SK031B518042


In [2]:
#ADD attribute "dmo_id" for each SGS
from modules import get_dmo_id_sgs_vgl
get_dmo_id_sgs_vgl.create_sgs_dmo_id()
print("dmo_id updated")

dmo_id updated


In [ ]:
list_alter_stmts = ['ALTER TABLE gtlab.sk_sgs_vgl_sguds ADD PRIMARY KEY (id);', 'ALTER TABLE gtlab.sk_sgs_vgl_sguds_details ADD PRIMARY KEY (id);', 'ALTER TABLE gtlab.sk_sgs_vgl_sguds ADD CONSTRAINT fk_sk_sgs_vgl_sguds_with_sk_dmos_raw  FOREIGN KEY (dmo_id) REFERENCES gtlab.sk_dmos_raw (dmo_id);', 'ALTER TABLE gtlab.sk_sgs_vgl_sguds ADD CONSTRAINT fk_sk_sgs_vgl_sguds_with_sk_municipalities_gku  FOREIGN KEY (gku_municipality_id) REFERENCES gtlab.sk_municipalities_gku ("Kód_obce");', 'ALTER TABLE gtlab.sk_sgs_vgl_sguds_details ADD CONSTRAINT fk_sk_sgs_vgl_sguds_with_sk_sgs_vgl_sguds_details  FOREIGN KEY (id) REFERENCES gtlab.sk_sgs_vgl_sguds (id);']

for stmt in list_alter_stmts:
    with engine.connect() as con1:
        con1.execute(text(stmt))
        con1.commit()
        con1.close()
        

In [4]:
list_alter_stmts = ["ALTER TABLE gtlab.sk_sgs_vgl_sguds ADD COLUMN topic_en text;", 
                    """

UPDATE gtlab.sk_sgs_vgl_sguds SET topic_en = nv.tema_en 
FROM (
SELECT objectid, 
	CASE 
		WHEN tema = 'vulkanity' THEN 'volcanites'
		WHEN tema = 'geomorfológia' THEN 'geomorphology'
		WHEN tema =  'archeológia' THEN 'archaeology'
		WHEN tema =  'hydrogeológia' THEN 'hydrogeology'
		WHEN tema =  'baníctvo' THEN 'mining'
		WHEN tema =  'mineralógia' THEN 'mineralogy'
		WHEN tema = 'všeobecná geológia' THEN 'general geology'
		WHEN tema =  'tektonika' THEN 'tectonics'
		WHEN tema =  'paleontológia' THEN 'paleontology'
		ELSE null
	END tema_en
FROM gtlab.sk_sgs_vgl_sguds
) nv
WHERE nv.objectid = sk_sgs_vgl_sguds.objectid;
"""
]

for stmt in list_alter_stmts:
    with engine.connect() as con1:
        con1.execute(text(stmt))
        con1.commit()
        con1.close()
        